# Harvest Extract (AWS)

Reading 6(six) PDF files with differing format of data.

## TODO: 

<span style="color:red; font-weight: bold; font-size: 50px;">
IMPLEMENT AWS TEXTRACT CODE !!
</span>



### Imports:

In [ ]:
from pandas import DataFrame, set_option
import pandas as pd
from pdfplumber import open
from os import listdir, path
from re import match, findall, compile, search, sub
from itertools import chain
from json import dump, dumps, load, loads
import traceback
from pdf2image import convert_from_path

In [128]:
def lines(location, start: int, end: int, heading: int, exclude: dict = []):
    collection: list = []
    pageNo: int = 1

    with open(location) as reader:
        for page in reader.pages:
            content: str = page.extract_text()
            lines: list = content.split("\n")

            if pageNo == 1:
                collection.extend(lines[slice(start, len(lines))])
                pageNo += 1
            else:
                collection.extend(lines[slice(heading, len(lines))])

    # for line in collection:
    #     print(line)

    return collection


def GetDataframe(rows, indexes: list):
    try:
        data = {}

        for y in range(len(rows)):
            for x in range(len(rows[y])):
                if y == 0:
                    if x in indexes:
                        data[rows[y][x]] = []
                else:
                    if x in indexes:
                        data[rows[0][x]].append(rows[y][x])

        # pd.from_dict(data)
        # pd.set_option('display.max_columns', 10)
        # pd.set_option('display.max_rows', 101)
        return DataFrame.from_dict(data)
    except Exception:
        print(traceback.format_exc())

    return None


def partition(line: str, segments: list):
    collection: list = []
    expression = (
        " "
        if len(segments) == 0
        else r"|".join([f"({segment})" for segment in segments])
    )
    pattern = compile(expression)

    if search(pattern, line):
        located: list = findall(pattern, line)
        for x in located:
            collection.extend([y.strip() for y in x if y != ""])

    return collection


def structure(lines: list):
    scale: dict = {}

    for line in lines:
        word: str = line[0]
        scale[word] = len(line)

    print(dumps(scale, indent=4))

def extract_text(path: str):
    images: list = convert_from_path(path)
    extract: Textractor = Textractor(profile_name='default')
    document = extractor.detect_document_text(file_source=images)

    print(document.lines)


## 2018_Bear_Harvest_By_Method_And_County.pdf

![image](.\Images\2018_Bear_Harvest_By_Method_And_County.png)

In [129]:
data = lines("./Documents/2018_Bear_Harvest_by_method_and_county.pdf", 1, -1, 3)
rows = []

for i in range(0, len(data)):
    if i == 0:
        rows.append(partition(data[i], ["Still", "Dog", "Percent Method"]))
    elif i == 1:
        rows.append(
            partition(
                data[i],
                ["Male", "Female", "County total", "County", "Total", "Still", "Dog"],
            )
        )
    else:
        rows.append(partition(data[i], ["^\w+", "\d+%", "\d+", "N/A[0-9]{1,}", "N/A"]))

### Data Extract

In [130]:
report = DataFrame(rows[slice(2, len(rows) - 1)], columns=rows[1])
report.head(n=43)

,County,Male,Female,Total,Male,Female,Total,County total,Still,Dog
0,Alamance,0,0,0,0,0,0,0,N/A1,N/A
1,Alexander,0,0,0,0,0,0,0,N/A,N/A
2,Alleghany,6,8,14,0,0,0,14,100%,0%
3,Anson,0,0,0,0,0,0,0,N/A,N/A
4,Ashe,19,16,35,2,0,2,37,95%,5%
5,Avery,5,11,16,19,17,36,52,31%,69%
6,Beaufort,19,38,57,82,55,137,194,29%,71%
7,Bertie,11,15,26,26,23,49,75,35%,65%
8,Bladen,18,21,39,31,25,56,95,41%,59%
9,Brunswick,5,6,11,11,10,21,32,34%,66%


## 2020-Turkey-Harvest-County.pdf

![image](.\Images\2020-Turkey-Harvest-County.png)

In [131]:
turkey_harvest_county = lines("./Documents/2020-Turkey-Harvest-County.pdf", 2, -1, 4)
collection: list = []

for i in range(len(turkey_harvest_county)):
    if i == 0:
        collection.append(
            partition(
                turkey_harvest_county[i], ["ADULT", "TOTAL", "GAME", "OTHER", "WRC"]
            )
        )
    elif i == 1:
        collection.append(
            partition(
                turkey_harvest_county[i],
                [
                    "COUNTY",
                    "GOBBLERS",
                    "JAKES",
                    "TURKEYS",
                    "LANDS",
                    "DISTRICT",
                    "REGION",
                ],
            )
        )
    else:
        collection.append(
            partition(turkey_harvest_county[i], ["^[\sA-Z]+", "[\d,]+", "\w+$"])
        )

map: dict = {}

for y in range(len(collection) - 1):
    if y == 0:
        for x in range(len(collection[y])):
            map[collection[y][x]] = []
    elif y == 1:
        for x in range(len(collection[y])):
            if x == 1:
                map[f"ADULT {collection[y][x]}"] = []
                del map["ADULT"]
            elif x == 3:
                map[f"TOTAL {collection[y][x]}"] = []
                del map["TOTAL"]
            elif x == 4:
                map[f"GAME {collection[y][x]}"] = []
                del map["GAME"]
            elif x == 5:
                map[f"OTHER {collection[y][x]}"] = []
                del map["OTHER"]
            elif x == 7:
                map[f"WRC {collection[y][x]}"] = []
                del map["WRC"]
            else:
                map[collection[y][x]] = []
    else:
        keys = list(map.keys())

        for x in range(len(collection[y])):
            map[keys[x]].append(collection[y][x].strip())

report = DataFrame.from_dict(map)

### Data Extract

In [132]:
report.head(n=60)

,COUNTY,ADULT GOBBLERS,JAKES,TOTAL TURKEYS,GAME LANDS,OTHER LANDS,DISTRICT,WRC REGION
0,ALAMANCE,143,28,171,0,171,5,CENTRAL
1,ALEXANDER,54,16,70,0,70,7,WESTERN
2,ALLEGHANY,134,31,165,1,164,7,WESTERN
3,ANSON,272,70,342,2,340,6,CENTRAL
4,ASHE,189,50,239,2,237,7,WESTERN
5,AVERY,98,25,123,12,111,8,WESTERN
6,BEAUFORT,284,79,363,6,357,2,COASTAL
7,BERTIE,400,34,434,20,414,1,COASTAL
8,BLADEN,484,87,571,29,542,4,COASTAL
9,BRUNSWICK,323,68,391,8,383,4,COASTAL


## 2022-2023-NC-Deer-Harvest-By-Weapon-Type.pdf

![image](.\Images\2022-2023-NC-Deer-Harvest-By-Weapon-Type.png)

In [133]:
nc_deer_harvest_by_weapon_type = lines(
    "./Documents/2022-2023-NC-Deer-Harvest-By-Weapon-Type.pdf", 1, -1, 3
)
collection: list = []

for i in range(len(nc_deer_harvest_by_weapon_type)):
    if i == 0:
        collection.append(
            partition(
                nc_deer_harvest_by_weapon_type[i],
                ["BOW", "CROSSBOW", "MUZZLELOADER", "GUN", "TOTAL"],
            )
        )
    elif i == 1:
        collection.append(
            partition(
                nc_deer_harvest_by_weapon_type[i],
                ["DISTRICT", "COUNTY", "ANTLERED BUCK", "BUTTON BUCK", "DOE", "TOTAL"],
            )
        )
    else:
        collection.append(
            partition(
                nc_deer_harvest_by_weapon_type[i],
                ["^\d", "[^\d]+[a-zA-Z]+\s[a-zA-Z]+[^\d]+", "[a-zA-Z]+", "[\d,]+"],
            )
        )

collection[len(collection) - 1].insert(0, "")

rows = collection[slice(1, len(collection))]

### Data Extract

In [134]:
report = DataFrame(collection[slice(2, len(collection))], columns=collection[1])
report.iloc[0:50, 0:23]

,DISTRICT,COUNTY,ANTLERED BUCK,BUTTON BUCK,DOE,TOTAL,ANTLERED BUCK,BUTTON BUCK,DOE,TOTAL,...,DOE,TOTAL,ANTLERED BUCK,BUTTON BUCK,DOE,TOTAL,ANTLERED BUCK,BUTTON BUCK,DOE,TOTAL
0,5,Alamance,108,5,109,222,74,10,63,147,...,45,178,842,52,726,"1,620","1,152",72,943,"2,167"
1,7,Alexander,27,4,43,74,17,4,20,41,...,39,174,441,36,326,803,618,46,428,"1,092"
2,7,Alleghany,49,0,47,96,52,2,46,100,...,95,350,646,51,589,"1,286",985,70,777,"1,832"
3,6,Anson,74,5,118,197,37,5,57,99,...,114,335,"1,681",185,"1,684","3,550","1,999",209,"1,973","4,181"
4,7,Ashe,99,14,103,216,84,6,75,165,...,123,558,907,93,762,"1,762","1,510",128,"1,063","2,701"
5,8,Avery,71,5,79,155,110,8,103,221,...,67,106,435,24,158,617,654,38,407,"1,099"
6,2,Beaufort,27,6,46,79,11,0,25,36,...,52,103,967,100,"1,126","2,193","1,049",113,"1,249","2,411"
7,1,Bertie,36,3,54,93,15,2,20,37,...,49,101,983,164,"1,240","2,387","1,083",172,"1,363","2,618"
8,4,Bladen,22,0,43,65,9,0,6,15,...,43,79,"1,162",110,"1,085","2,357","1,224",115,"1,177","2,516"
9,4,Brunswick,41,5,81,127,15,3,14,32,...,24,74,740,59,742,"1,541",844,69,861,"1,774"


## Harv_GL18_1.pdf

![image](.\Images\Harv_GL18_1.png)

In [135]:
harv_gl18_1 = lines('.\Documents\Harv_GL18_1.pdf', 1, -1, 2)
collection: list = []

for i in range(len(harv_gl18_1)):
    if i == 0:
        collection.append(partition(harv_gl18_1[i], ['GAME LAND', 'ANTLERED BUCK', 'BUTTON BUCK', 'DOE', 'TOTAL']))
    else:
        collection.append(partition(harv_gl18_1[i], ['^[\sa-zA-Z.-]+', '[^ ]+']))

report = GetDataframe(collection, [0, 1, 2, 3, 4])

### Data Extract

In [136]:
report.head(n=53)

,GAME LAND,ANTLERED BUCK,BUTTON BUCK,DOE,TOTAL
0,Alcoa,66,14,97,177
1,Alligator River,5,2,4,11
2,Angola Bay,16,1,16,33
3,Bachelor Bay,0,0,2,2
4,Bertie County,22,0,15,37
5,Bladen Lakes State Forest,32,4,26,62
6,Brinkleyville,1,0,1,2
7,Brunswick County,0,0,0,0
8,Buckhorn,3,0,0,3
9,Buckridge,5,1,5,11


## Bear-Harvest-by-Game-Land-2008-2022.pdf

![image](.\Images\Bear-Harvest-by-Game-Land-2008-2022.png)


In [137]:
bear_harvest_by_game_land_2008_2022 = lines('.\Documents\Bear-Harvest-by-Game-Land-2008-2022.pdf', 3, -1, 2)
collection: list = []

for i in range(len(bear_harvest_by_game_land_2008_2022)):
    if i == 2:
        collection.append(partition(bear_harvest_by_game_land_2008_2022[i], ['Game Land', 'Management Unit', '\d{4}', 'Total']))
    else:
        collection.append(partition(bear_harvest_by_game_land_2008_2022[i], ['^[ .\-/*a-zA-Z]+', '[\d,]+', '[ a-zA-Z]+$']))

pattern = compile(r'Coastal BMU|Mountain BMU|Piedmont BMU|No Season|n/a')
missing_space_pattern = compile(r'Other lands/Private/Unknown')

for i in range(len(collection)):

    l = collection[i]

    if len(l) > 0 and search(pattern, l[0]):
        beacon = findall(pattern, l[0])
        v = [sub(pattern, '', l[0]).strip()]
        v.extend(beacon)
        v.extend(l[slice(1, len(l))])
        collection[i] = v
    elif search(missing_space_pattern, l[0]):
        v = [l[0].strip()]
        v.extend([' '])
        v.extend(l[slice(1, len(l))])
        collection[i] = v

map: dict = {}

for y in range(1, len(collection) - 2):
    if y == 1:
        for x in range(len(collection[y])):
            map[collection[y][x]] = []
    if y == 2:
        for x in range(len(collection[y])):
            if x == 1:
                map[f'Bear {collection[y][x]}'] = []
                del map['Bear']
            else:
                map[collection[y][x]] = []
    else:
        keys = list(map.keys())

        for x in range(len(collection[y])):
            map[keys[x]].append(collection[y][x].strip())

data = DataFrame.from_dict(map)

### Data Extract

In [138]:
data.iloc[0:54, 0:18]

,Game Land,Bear Management Unit,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total
0,Alligator River,Coastal BMU,1,11,12,6,9,8,11,14,10,4,3,8,6,10,7,120
1,Angola Bay,Coastal BMU,2,2,6,6,3,4,2,3,1,8,7,7,8,3,3,65
2,Bachelor Bay,Coastal BMU,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Bertie County*,Coastal BMU,0,2,0,1,1,0,1,0,1,1,1,0,3,1,1,13
4,Bladen Lakes State Forest,Coastal BMU,2,9,15,9,7,5,9,16,6,10,4,7,9,8,7,123
5,Buckridge,Coastal BMU,9,9,16,19,22,12,12,18,11,19,4,14,21,19,21,226
6,Cape Fear River Wetlands,Coastal BMU,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3
7,Carteret County*,Coastal BMU,3,0,1,1,2,1,0,1,1,0,1,1,3,0,0,15
8,Chowan Swamp,Coastal BMU,1,5,5,6,10,4,4,3,2,5,4,6,8,8,9,80
9,Columbus County,Coastal BMU,0,3,0,0,0,0,2,0,1,0,1,0,1,1,0,9
